In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
idx = ('Soil', 'Lake')

In [4]:
def do_optim(ds, ID, soil, depo, lake, printout=True) :

    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ExCa'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['ExMg'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ExNa'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['ExK'])
    
    
    params_a, comparisons_a = magicopt.get_acid_anion_setup(ds, idx, 'Observed')
    
    res_pars1, nfev1 = magicopt.calib(ds, params_a, comparisons_a, method='nelder')# method='leastsq')
    
    
    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ECa-0'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['EMg-0'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ENa-0'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['EK-0'])
    
    params, comparisons = magicopt.get_base_cations_setup(ds, 
                            idx, 'Observed', usemax=True, obsE=True)
    
    #params.pretty_print()
    #Set initial guess to the present-day value
    params['ECa_S'].value = soil['ExCa']+20
    params['EMg_S'].value = soil['ExMg']
    params['ENa_S'].value = soil['ExNa']
    params['EK_S'].value = soil['ExK']
    
    params['WCa_S'].value*=0.2
    params['WMg_S'].value*=0.2
    params['WNa_S'].value*=0.2
    params['WK_S'].value*=0.2
    
    params['WK_S'].min = max(0.0, soil['UptK'] - depo['K'])
    
    
    ntries = 0
    err=0
    abserr=0
    
    skip = True
    
    while (abserr>1 and ntries < 10) or ntries==0 :
        if err < 0. :
            params['WCa_S'].value*=2
            if ID==3096:
                params['ECa_S'].value+=5
        elif err > 0.:
            params['WCa_S'].value/=2
            if ID==3096:
                params['ECa_S'].value-=5

        res_pars2, nfev = magicopt.calib(ds, params, comparisons, 
                                    #method='nelder'
                                    method='leastsq'
                                    )

        sim = ds.get_result_series('Ca(2+) ionic concentration', ['Lake'])
        obs = ds.get_input_series('Observed Ca', [], alignwithresults=True)
        err = np.nansum(sim-obs)
        abserr = np.nansum(np.abs(sim-obs))

        ntries += 1
        if skip : break
    
    #if np.abs(err)>1.:
    #    high_error_ids.append((ID,err))
    #    print('High Ca error (abs>1)')

    if printout :
        comp = comparisons
        comp += comparisons_a
        comp += [('SO4(2-) ionic concentration', ['Lake'], 'Observed SO4', [])]

        for comparison in comp :
            simname, simindexes, obsname, obsindexes, *_ = comparison

            sim = ds.get_result_series(simname, simindexes)
            obs = ds.get_input_series(obsname, obsindexes, alignwithresults=True)

            err = np.sqrt(np.nansum(np.square(sim-obs)))

            print('Error of %s: RSSE(sim-obs): %f' % (obsname, err))

        print('\n')
        res_pars1.pretty_print()
        res_pars2.pretty_print()
        print('\n')
        
        ds.write_parameters_to_file('MobiusFiles/params_%d.dat'%ID)
        ds.write_inputs_to_file('MobiusFiles/inputs_%d.dat'%ID)
    
    return res_pars1, res_pars2


def do_fuzzy(ds, ID, soil, depo, lake):
    
    nruns = 10
    
    fuzzy_params = [
        ('Observed NO3', [], 0.9, 1.1, 'obs'),
        ('Observed Ca', [], 0.9, 1.1, 'obs'),
        ('Observed Mg', [], 0.9, 1.1, 'obs'),
        ('Observed Na', [], 0.9, 1.1, 'obs'),
        ('Observed K', [], 0.9, 1.1, 'obs'),
        ('Depth', ['Soil'], 0.9, 1.1, 'par'),
    ]
    
    multipliers = np.zeros((len(fuzzy_params), nruns))
    values      = np.zeros((len(fuzzy_params), nruns))
    
    respars1 = []
    respars2 = []
    
    for idx, par in enumerate(fuzzy_params) :
        name, index, minimum, maximum, partype = par
        multipliers[idx, :] = np.random.uniform(minimum, maximum, nruns)
        
    for run in range(nruns) :
        ds2 = ds.copy()
        for idx, par in enumerate(fuzzy_params):
            name, index, minimum, maximum, partype = par
            if partype == 'obs':
                values = ds2.get_input_series(name, index)
                values *= multipliers[idx, run]
                ds2.set_input_series(name, index, values)
            elif partype == 'par':
                value = ds2.get_parameter_double(name, index)
                value *= multipliers[idx, run]
                ds2.set_parameter_double(name, index, value)
        res1,res2 = do_optim(ds2, ID, soil, depo, lake, printout=True)
        
        respars1.append(res1)
        respars2.append(res2)
        
        ds2.delete()
    
    #TODO: We need to store them somehow instead
    #return fuzzy_params, multipliers, respars1, respars2

In [6]:
#soilfile, lakefile1, lakefile2, depofile, seqfile = excelfiles
excelfiles = ('CCE-soil BJC.xls', 'CCE-lake BJC.xls', 'WRI-lake 2019.xls', 'CCE-depo BJC.xls', 'CCE-seqs BJC.xls')

#multi.do_magic_loop(dataset, excelfile, do_fuzzy, do_id=9)
multi.do_magic_loop(dataset, excelfiles, do_optim, two_year=False, limit_num = 10)#, do_id=9)


#print('IDs with error in Ca > 1: ', high_error_ids)

Running lake Hokksjøen (ID 9)

Error of Observed Ca: RSSE(sim-obs): 0.006465
Error of Observed Mg: RSSE(sim-obs): 0.004706
Error of Observed Na: RSSE(sim-obs): 0.002437
Error of Observed K: RSSE(sim-obs): 0.000219
Error of Observed ECa: RSSE(sim-obs): 0.001520
Error of Observed EMg: RSSE(sim-obs): 0.000539
Error of Observed ENa: RSSE(sim-obs): 0.000112
Error of Observed EK: RSSE(sim-obs): 0.000219
Error of Observed NO3: RSSE(sim-obs): 6.039016
Error of Observed SO4: RSSE(sim-obs): 1.924601


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     25.46     0.01    88.54 0.003463     True     None     None
EK_S      5.962     0.01       15 0.0007691     True     None     None
EMg_S     11.51     0.01    36.41 0.001483     True     None     None
ENa_S      5.44     0.01    13.08 0.000757     True     None     None


Error of Observed Ca: RSSE(sim-obs): 0.000004
Error of Observed Mg: RSSE(sim-obs): 0.000001
Error of Observed Na: RSSE(sim-obs): 0.000026
Error of Observed K: RSSE(sim-obs): 0.000000
Error of Observed ECa: RSSE(sim-obs): 0.000004
Error of Observed EMg: RSSE(sim-obs): 0.000001
Error of Observed ENa: RSSE(sim-obs): 0.000002
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 0.149154
Error of Observed SO4: RSSE(sim-obs): 1.067054


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S      17.9     0.01    84.58 1.82e-05     True     None     None
EK_S      2.439     0.01    9.797 2.503e-06     True     None     None
EMg_S     3.639     0.01    24.07 3.69e-06     True     None     None
ENa_S     1.951     0.01    8.459 1.953e-06     True     None     None
WCa_S      12.4        0    17